## Dataset: Factores de Emisión del Inventario Nacional de Emisiones Atmosféricas

**Archivo:** `Factores_de_emision.csv`

Este dataset contiene información oficial sobre **factores de emisión** asociados a diferentes actividades, combustibles y procesos industriales. Los factores de emisión permiten estimar la cantidad de contaminantes emitidos a la atmósfera por unidad de actividad (por ejemplo: kg de CO₂ por litro de combustible quemado).

### Contenido típico del dataset
Incluye variables como:

- **Actividad / Fuente** (e.g., combustión móvil, industria, sector residencial).  
- **Tipo de contaminante** (CO₂, CO, NOₓ, PM2.5, PM10, etc.).  
- **Combustible o insumo asociado** (diésel, gas natural, gasolina, biomasa…).  
- **Factor de emisión** (valor numérico que indica la cantidad emitida por unidad de actividad).  
- **Unidades del factor** (e.g., kg/TJ, g/kWh, kg/tonelada).  
- **Año, región o metodología**, dependiendo de la fuente.

In [2]:
# Importar las librerías
import pandas as pd
import numpy as np
from pathlib import Path
import re

DATA_DIR = Path("../data")

def load_data():
    """Carga el dataset."""
    ile_path = DATA_DIR / "raw" / "Factores_de_emision_reducido.csv.gz"
    return pd.read_csv(ile_path, compression='gzip')

df = load_data()
df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_20736\2169266291.py:12: DtypeWarning: Columns (2,5,8,12,19,21,22,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(ile_path, compression='gzip')


,ANO,REGION,DESAGREGACION_ESPECIFICA,CARTERA,ENTIDAD,INSTRUMENTO,FUENTE_ESPECIFICA,S,S*,C,...,CLASIFICACION2,CLASIFICACION3,NOMBRE_FACTOR,NIVEL,CODIGO_FACTOR,VALOR_F,UNIDAD_F,INC_INFERIOR,INC_SUPERIOR,CONTAMINANTE
0,"1,990",NaN,NaN,Minenergía,UPME,FECOC 2016,FECOC - UPME nombre del archivo FECOC-UPME_EF_...,1.0,NaN,A,...,CENTRALES TERMICAS,NaN,CONSUMO DE COMBUSTIBLE EN GENERACION DE ENERGIA,NaN,1990NACIONAL1A1aiCARBON MINERALCENTRALES TERMI...,"88,136",kg CO2/TJ,0.0,0.0,CO2
1,"1,990",NaN,NaN,Minenergía,UPME,FECOC 2016,FECOC - UPME nombre del archivo FECOC-UPME_EF_...,1.0,NaN,A,...,CENTRALES TERMICAS,NaN,CONSUMO DE COMBUSTIBLE EN GENERACION DE ENERGIA,NaN,1990NACIONAL1A1aiDIESEL OILCENTRALES TERMICASC...,"74,193,483",kg CO2/TJ,0.0,0.0,CO2
2,"1,990",NaN,NaN,Minenergía,UPME,FECOC 2016,FECOC - UPME nombre del archivo FECOC-UPME_EF_...,1.0,NaN,A,...,CENTRALES TERMICAS,NaN,CONSUMO DE COMBUSTIBLE EN GENERACION DE ENERGIA,NaN,1990NACIONAL1A1aiFUEL OILCENTRALES TERMICASCON...,"78,281,203",kg CO2/TJ,0.0,0.0,CO2
3,"1,990",NaN,NaN,Minenergía,UPME,FECOC 2016,FECOC - UPME nombre del archivo FECOC-UPME_EF_...,1.0,NaN,A,...,CENTRALES TERMICAS,NaN,CONSUMO DE COMBUSTIBLE EN GENERACION DE ENERGIA,NaN,1990NACIONAL1A1aiGAS NATURALCENTRALES TERMICAS...,"55,539.11",kg CO2/TJ,0.0,0.0,CO2
4,"1,990",NaN,NaN,Minenergía,UPME,FECOC 2016,FECOC - UPME nombre del archivo FECOC-UPME_EF_...,1.0,NaN,A,...,CENTRALES TERMICAS,NaN,CONSUMO DE COMBUSTIBLE EN GENERACION DE ENERGIA,NaN,1990NACIONAL1A1aiQUEROSENECENTRALES TERMICASCO...,"73,939,639",kg CO2/TJ,0.0,0.0,CO2


In [3]:
# Información del dataset
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226381 entries, 0 to 226380
Data columns (total 31 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ANO                       222232 non-null  object 
 1   REGION                    143454 non-null  object 
 2   DESAGREGACION_ESPECIFICA  5056 non-null    object 
 3   CARTERA                   222693 non-null  object 
 4   ENTIDAD                   201324 non-null  object 
 5   INSTRUMENTO               56914 non-null   object 
 6   FUENTE_ESPECIFICA         161543 non-null  object 
 7   S                         223114 non-null  float64
 8   S*                        125760 non-null  object 
 9   C                         223114 non-null  object 
 10  SB1                       220106 non-null  float64
 11  SB2                       208865 non-null  object 
 12  SB3                       41283 non-null   object 
 13  SB4                       13198 non-null   f

Las variables a usar serán las siguientes:

- SECTOR_IPCC

- CATEGORIA

- SUBCATEGORIA1

- SUBCATEGORIA2

- NOMBRE_FACTOR

- CONTAMINANTE

- UNIDAD_F

- ANO

- CODIGO_FACTOR

Las mencionadas aportan valor, no son redudantes y además es facil convertirlas a variables categoricas.

In [4]:
# Eliminar columnas innecesarias
cols_to_drop = [
    'DESAGREGACION_ESPECIFICA','CARTERA','INSTRUMENTO',
    'FUENTE_ESPECIFICA','S','S*','C','SB1','SB2','SB3','SB4',
    'SUBCATEGORIA3','SUBCATEGORIA4','CLASIFICACION1','CLASIFICACION2',
    'CLASIFICACION3','NIVEL','INC_INFERIOR','INC_SUPERIOR'
]

df = df.drop(columns=cols_to_drop)

In [5]:
# Filtrar por contaminante CO2
df_v2 = df[df['CONTAMINANTE'] == 'CO2'].reset_index(drop=True)

# Convertir año a entero
df_v2["ANO"] = pd.to_numeric(df_v2["ANO"].str.replace(",", "", regex=False), errors="coerce")
df_v2.dropna(subset=["ANO"], inplace=True)
df_v2["ANO"] = df_v2["ANO"].astype(int)


# Convertir Valor_F a flotante
df_v2["VALOR_F"] = df_v2["VALOR_F"].astype(str).str.replace(",", "").astype(float)

df_v2.head()


,ANO,REGION,ENTIDAD,SECTOR_IPCC,CATEGORIA,SUBCATEGORIA1,SUBCATEGORIA2,NOMBRE_FACTOR,CODIGO_FACTOR,VALOR_F,UNIDAD_F,CONTAMINANTE
0,1990,NaN,UPME,1. Energía,1.A. Actividades de quema de combustibles,1.A.1. Industrias de la energía,1.A.1.a. Producción de electricidad pública y ...,CONSUMO DE COMBUSTIBLE EN GENERACION DE ENERGIA,1990NACIONAL1A1aiCARBON MINERALCENTRALES TERMI...,88136.00,kg CO2/TJ,CO2
1,1990,NaN,UPME,1. Energía,1.A. Actividades de quema de combustibles,1.A.1. Industrias de la energía,1.A.1.a. Producción de electricidad pública y ...,CONSUMO DE COMBUSTIBLE EN GENERACION DE ENERGIA,1990NACIONAL1A1aiDIESEL OILCENTRALES TERMICASC...,74193483.00,kg CO2/TJ,CO2
2,1990,NaN,UPME,1. Energía,1.A. Actividades de quema de combustibles,1.A.1. Industrias de la energía,1.A.1.a. Producción de electricidad pública y ...,CONSUMO DE COMBUSTIBLE EN GENERACION DE ENERGIA,1990NACIONAL1A1aiFUEL OILCENTRALES TERMICASCON...,78281203.00,kg CO2/TJ,CO2
3,1990,NaN,UPME,1. Energía,1.A. Actividades de quema de combustibles,1.A.1. Industrias de la energía,1.A.1.a. Producción de electricidad pública y ...,CONSUMO DE COMBUSTIBLE EN GENERACION DE ENERGIA,1990NACIONAL1A1aiGAS NATURALCENTRALES TERMICAS...,55539.11,kg CO2/TJ,CO2
4,1990,NaN,UPME,1. Energía,1.A. Actividades de quema de combustibles,1.A.1. Industrias de la energía,1.A.1.a. Producción de electricidad pública y ...,CONSUMO DE COMBUSTIBLE EN GENERACION DE ENERGIA,1990NACIONAL1A1aiQUEROSENECENTRALES TERMICASCO...,73939639.00,kg CO2/TJ,CO2


In [6]:

def llenar_region(texto):
    texto = str(texto).upper()
    for clave, region in regiones.items():
        if clave in texto:
            return region
    return None

def llenar_departamento(texto):
    texto = str(texto).upper()
    for clave, (region, depto) in departamentos.items():
        if clave in texto:
            return depto
    return None

def llenar_region_depto(texto):
    texto = str(texto).upper()
    for clave, (region, depto) in departamentos.items():
        if clave in texto:
            return region
    return None

regiones = {
    
    "CARIBE": "CARIBE",
    "REGION CARIBE": "CARIBE",

    "PACIFICA": "PACIFICA",
    "PACÍFICA": "PACIFICA",

    "ANDINA": "ANDINA",

    "AMAZONIA": "AMAZONIA",
    "AMAZONÍA": "AMAZONIA",

    "ORINOQUIA": "ORINOQUIA",
    "ORINOQUÍA": "ORINOQUIA",

    "NACIONAL": "NACIONAL"

}

departamentos = {

    # Departamentos
    "AMAZONAS": ("AMAZONIA", "AMAZONAS"),
    "ANTIOQUIA": ("ANDINA", "ANTIOQUIA"),

    "ARAUCA": ("ORINOQUIA", "ARAUCA"),

    "ATLANTICO": ("CARIBE", "ATLÁNTICO"),
    "ATLÁNTICO": ("CARIBE", "ATLÁNTICO"),

    "BOLIVAR": ("CARIBE", "BOLÍVAR"),
    "BOLÍVAR": ("CARIBE", "BOLÍVAR"),

    "BOYACA": ("ANDINA", "BOYACÁ"),
    "BOYACÁ": ("ANDINA", "BOYACÁ"),

    "CALDAS": ("ANDINA", "CALDAS"),

    "CAQUETA": ("AMAZONIA", "CAQUETÁ"),
    "CAQUETÁ": ("AMAZONIA", "CAQUETÁ"),

    "CASANARE": ("ORINOQUIA", "CASANARE"),

    "CAUCA": ("PACIFICA", "CAUCA"),

    "CESAR": ("CARIBE", "CESAR"),

    "CHOCO": ("PACIFICA", "CHOCÓ"),
    "CHOCÓ": ("PACIFICA", "CHOCÓ"),

    "CORDOBA": ("CARIBE", "CÓRDOBA"),
    "CÓRDOBA": ("CARIBE", "CÓRDOBA"),

    "CUNDINAMARCA": ("ANDINA", "CUNDINAMARCA"),

    "GUAINIA": ("AMAZONIA", "GUAINÍA"),
    "GUAINÍA": ("AMAZONIA", "GUAINÍA"),

    "GUAJIRA": ("CARIBE", "LA GUAJIRA"),
    "LA GUAJIRA": ("CARIBE", "LA GUAJIRA"),

    "GUAVIARE": ("AMAZONIA", "GUAVIARE"),

    "HUILA": ("ANDINA", "HUILA"),

    "MAGDALENA": ("CARIBE", "MAGDALENA"),

    "META": ("ORINOQUIA", "META"),

    "NARIÑO": ("PACIFICA", "NARIÑO"),

    "NORTE DE SANTANDER": ("ANDINA", "NORTE DE SANTANDER"),
    "NORTE SANTANDER": ("ANDINA", "NORTE DE SANTANDER"),

    "PUTUMAYO": ("AMAZONIA", "PUTUMAYO"),

    "QUINDIO": ("ANDINA", "QUINDÍO"),
    "QUINDÍO": ("ANDINA", "QUINDÍO"),

    "RISARALDA": ("ANDINA", "RISARALDA"),

    "SAN ANDRES": ("CARIBE", "SAN ANDRÉS"),
    "SAN ANDRÉS": ("CARIBE", "SAN ANDRÉS"),

    "SANTANDER": ("ANDINA", "SANTANDER"),

    "SUCRE": ("CARIBE", "SUCRE"),

    "TOLIMA": ("ANDINA", "TOLIMA"),

    "VALLE": ("PACIFICA", "VALLE DEL CAUCA"),
    "VALLE DEL CAUCA": ("PACIFICA", "VALLE DEL CAUCA"),

    "VAUPES": ("AMAZONIA", "VAUPÉS"),
    "VAUPÉS": ("AMAZONIA", "VAUPÉS"),

    "VICHADA": ("ORINOQUIA", "VICHADA")
}



In [7]:
# Convertir columnas a mayúsculas
df_v2["CODIGO_FACTOR"] = df_v2["CODIGO_FACTOR"].astype(str).str.upper()
df_v2["ENTIDAD"] = df_v2["ENTIDAD"].astype(str).str.upper()
df_v2["REGION"] = df_v2["REGION"].astype(str).str.upper()

df_v2.head()


,ANO,REGION,ENTIDAD,SECTOR_IPCC,CATEGORIA,SUBCATEGORIA1,SUBCATEGORIA2,NOMBRE_FACTOR,CODIGO_FACTOR,VALOR_F,UNIDAD_F,CONTAMINANTE
0,1990,NAN,UPME,1. Energía,1.A. Actividades de quema de combustibles,1.A.1. Industrias de la energía,1.A.1.a. Producción de electricidad pública y ...,CONSUMO DE COMBUSTIBLE EN GENERACION DE ENERGIA,1990NACIONAL1A1AICARBON MINERALCENTRALES TERMI...,88136.00,kg CO2/TJ,CO2
1,1990,NAN,UPME,1. Energía,1.A. Actividades de quema de combustibles,1.A.1. Industrias de la energía,1.A.1.a. Producción de electricidad pública y ...,CONSUMO DE COMBUSTIBLE EN GENERACION DE ENERGIA,1990NACIONAL1A1AIDIESEL OILCENTRALES TERMICASC...,74193483.00,kg CO2/TJ,CO2
2,1990,NAN,UPME,1. Energía,1.A. Actividades de quema de combustibles,1.A.1. Industrias de la energía,1.A.1.a. Producción de electricidad pública y ...,CONSUMO DE COMBUSTIBLE EN GENERACION DE ENERGIA,1990NACIONAL1A1AIFUEL OILCENTRALES TERMICASCON...,78281203.00,kg CO2/TJ,CO2
3,1990,NAN,UPME,1. Energía,1.A. Actividades de quema de combustibles,1.A.1. Industrias de la energía,1.A.1.a. Producción de electricidad pública y ...,CONSUMO DE COMBUSTIBLE EN GENERACION DE ENERGIA,1990NACIONAL1A1AIGAS NATURALCENTRALES TERMICAS...,55539.11,kg CO2/TJ,CO2
4,1990,NAN,UPME,1. Energía,1.A. Actividades de quema de combustibles,1.A.1. Industrias de la energía,1.A.1.a. Producción de electricidad pública y ...,CONSUMO DE COMBUSTIBLE EN GENERACION DE ENERGIA,1990NACIONAL1A1AIQUEROSENECENTRALES TERMICASCO...,73939639.00,kg CO2/TJ,CO2


In [8]:
# Crear departamento desde CODIGO_FACTOR

df_v2['REGION'] = df_v2['REGION'].replace(["NaN", "nan", "None", "","NAN"], np.nan)
df_v2["REGION"] = df_v2["REGION"].where(
    df_v2["REGION"].notna(),
    df_v2['CODIGO_FACTOR'].apply(lambda x: llenar_region(x))
)

df_v2["DEPARTAMENTO"] = df_v2["CODIGO_FACTOR"].apply(lambda x: llenar_departamento(x))

df_v2["DEPARTAMENTO"] = df_v2["DEPARTAMENTO"].where(
    df_v2["DEPARTAMENTO"].notna(),
    df_v2['ENTIDAD'].apply(lambda x: llenar_departamento(x))
)

df_v2["REGION"] = df_v2["REGION"].where(
    df_v2["REGION"].notna(),
    df_v2["DEPARTAMENTO"].apply(lambda x: llenar_region_depto(x))
)

df_v2.dropna(inplace=True)

In [9]:
# Veamos la cantidad de años y departamentos únicos
print("Años únicos:", df_v2["ANO"].nunique())
print("Departamentos únicos:", df_v2["DEPARTAMENTO"].nunique())
print("Regiones únicas:", df_v2["REGION"].nunique())

Años únicos: 44
Departamentos únicos: 31
Regiones únicas: 5


In [10]:
df_v2.dropna(inplace=True)

df_v2["REGION"].unique()

array(['ANDINA', 'ORINOQUIA', 'PACIFICA', 'CARIBE', 'AMAZONIA'],
      dtype=object)

In [11]:
df_v2["DEPARTAMENTO"].unique()

array(['ANTIOQUIA', 'BOYACÁ', 'CASANARE', 'CAUCA', 'CUNDINAMARCA',
       'NORTE DE SANTANDER', 'CESAR', 'CÓRDOBA', 'LA GUAJIRA',
       'SANTANDER', 'HUILA', 'CALDAS', 'AMAZONAS', 'ARAUCA', 'ATLÁNTICO',
       'BOLÍVAR', 'CAQUETÁ', 'TOLIMA', 'CHOCÓ', 'GUAINÍA', 'GUAVIARE',
       'MAGDALENA', 'META', 'SAN ANDRÉS', 'NARIÑO', 'PUTUMAYO', 'QUINDÍO',
       'RISARALDA', 'SUCRE', 'VAUPÉS', 'VICHADA'], dtype=object)

In [12]:
df_v2.shape[0]

13164

In [13]:
df_v2.head()

,ANO,REGION,ENTIDAD,SECTOR_IPCC,CATEGORIA,SUBCATEGORIA1,SUBCATEGORIA2,NOMBRE_FACTOR,CODIGO_FACTOR,VALOR_F,UNIDAD_F,CONTAMINANTE,DEPARTAMENTO
8773,1990,ANDINA,IPCC/CONSULTA EXPERTOS,1. Energía,1.B. Emisiones fugitivas de combustibles,1.B.1. Combustibles sólidos,1.B.1.a. Minería carbonífera y manejo del carbón.,PRODUCCION DE CARBON EN MINAS SUBTERRANEAS,1990CUENCA ANTIOQUIANA1B1AI M0MINERIA SUBTERRA...,5.9,m3 CO2/t,CO2,ANTIOQUIA
8774,1990,ANDINA,IPCC/CONSULTA EXPERTOS,1. Energía,1.B. Emisiones fugitivas de combustibles,1.B.1. Combustibles sólidos,1.B.1.a. Minería carbonífera y manejo del carbón.,PRODUCCION DE CARBON EN MINAS SUBTERRANEAS,1990CUENCA BOYACANA1B1AI M0MINERIA SUBTERRANEA...,5.9,m3 CO2/t,CO2,BOYACÁ
8775,1990,ORINOQUIA,IPCC/CONSULTA EXPERTOS,1. Energía,1.B. Emisiones fugitivas de combustibles,1.B.1. Combustibles sólidos,1.B.1.a. Minería carbonífera y manejo del carbón.,PRODUCCION DE CARBON EN MINAS SUBTERRANEAS,1990CUENCA CASANARENA1B1AI M0MINERIA SUBTERRAN...,5.9,m3 CO2/t,CO2,CASANARE
8776,1990,PACIFICA,IPCC/CONSULTA EXPERTOS,1. Energía,1.B. Emisiones fugitivas de combustibles,1.B.1. Combustibles sólidos,1.B.1.a. Minería carbonífera y manejo del carbón.,PRODUCCION DE CARBON EN MINAS SUBTERRANEAS,1990CUENCA CAUCANA1B1AI M0MINERIA SUBTERRANEAM...,5.9,m3 CO2/t,CO2,CAUCA
8777,1990,ANDINA,IPCC/CONSULTA EXPERTOS,1. Energía,1.B. Emisiones fugitivas de combustibles,1.B.1. Combustibles sólidos,1.B.1.a. Minería carbonífera y manejo del carbón.,PRODUCCION DE CARBON EN MINAS SUBTERRANEAS,1990CUENCA CUNDINAMARCANA1B1AI M0MINERIA SUBTE...,5.9,m3 CO2/t,CO2,CUNDINAMARCA


In [15]:
df_v2.columns.tolist()

['ANO',
 'REGION',
 'ENTIDAD',
 'SECTOR_IPCC',
 'CATEGORIA',
 'SUBCATEGORIA1',
 'SUBCATEGORIA2',
 'NOMBRE_FACTOR',
 'CODIGO_FACTOR',
 'VALOR_F',
 'UNIDAD_F',
 'CONTAMINANTE',
 'DEPARTAMENTO']

In [ ]:
# df_v2.to_csv(DATA_DIR / "processed" / "Factores_de_emision_CO2.csv", index=False)